In [ ]:
import numpy as np
import pandas as pd
import math
import os
import sys
import glob
import time
import pickle
import time
import itertools
import altair as alt
alt.data_transformers.disable_max_rows()

from astropy import units as u
from astropy.timeseries import LombScargle

import fastdtw
from scipy.spatial.distance import euclidean

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import rc
plt.style.use('classic')
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
rc('figure', facecolor='w')
rc('xtick', labelsize=20)
rc('ytick', labelsize=20)

import multiprocessing as mp
from multiprocessing import Pool
print("Number of processors: ", mp.cpu_count())

sys.path.append('/astro/users/jbirky/projects/tess_binaries')
os.environ['TESS_DATA'] = '/data/epyc/projects2/tess'

import tess_binaries as tb

In [ ]:
tess_xm = pd.read_csv(tb.cat_dir + '/asassn_tess_xm.csv.gz')
psamp = tess_xm[~np.isnan(tess_xm['period'])]
ref = psamp[psamp['period'] < 28]

In [ ]:
tsteps = 50
tarr = np.arange(0,tsteps,1)
pharr = np.linspace(0,1,tsteps)

farr = []
ids = []
periods = []
types = []

# demo = [5,16, 20,25, 4,6, 14,21, 62,117]
demo = [5,20]

for i in demo:
    tic_id = list(ref['tic_id'])[i]
    sec = list(ref['sector'])[i]
    typ = list(ref['Type'])[i]
    per = list(ref['period'])[i]
    print(i, typ)
    
    data_full = tb.readSourceFiles(tic_id, sector=sec)[0]  
    data      = data_full.fold(period=per*u.day) 
    bin_flux  = tb.binData(data, tsteps)
    
    tbins = np.linspace(min(data.time.jd), max(data.time.jd), tsteps+1)
    bin_width = (tbins[1] - tbins[0])/2
    
    plt.figure(figsize=[16,8])
    plt.ticklabel_format(useOffset=False)
    plt.scatter(data.time.jd, data['pdcsap_flux']/np.nanmedian(data['pdcsap_flux']), \
             color='k', s=2, label=f"{typ}: TIC\_{tic_id}\nP={per}")
    plt.plot(tbins[:tsteps]+bin_width, bin_flux, color='r', label=f'Bins: {tsteps}')

    plt.ylabel('PDCSAP Flux', fontsize=18)
    plt.xlabel('Julian Date', fontsize=18)
    plt.xlim(min(data.time.jd), max(data.time.jd))
    plt.legend(loc='lower right', fontsize=16)
    plt.minorticks_on()
    plt.show() 
    
    farr.append(bin_flux)
    ids.append(tic_id)
    periods.append(per)
    types.append(typ)

In [ ]:
int DTWDistance(s: array [1..n], t: array [1..m]) {
   DTW := array [0..n, 0..m]
 
   for i := 1 to n
     for j := 1 to m
       DTW[i, j] := infinity
   DTW[0, 0] := 0
 
   for i := 1 to n
       for j := 1 to m
           cost := d(s[i], t[j])
           DTW[i, j] := cost + minimum(DTW[i-1, j  ],    // insertion
                                       DTW[i  , j-1],    // deletion
                                       DTW[i-1, j-1])    // match
 
   return DTW[n, m]
}

int DTWDistance(s: array [1..n], t: array [1..m], w: int) {
    DTW := array [0..n, 0..m]

    w := max(w, abs(n-m)) // adapt window size (*)

    for i := 0 to n
        for j:= 0 to m
            DTW[i, j] := infinity
    DTW[0, 0] := 0
    for i := 1 to n
        for j := max(1, i-w) to min(m, i+w)
            DTW[i, j] := 0

    for i := 1 to n
        for j := max(1, i-w) to min(m, i+w)
            cost := d(s[i], t[j])
            DTW[i, j] := cost + minimum(DTW[i-1, j  ],    // insertion
                                        DTW[i  , j-1],    // deletion
                                        DTW[i-1, j-1])    // match

    return DTW[n, m]
}

In [ ]:
def euclidean(x, y):
#     return np.sqrt(x**2 + y**2)
    return np.abs(x-y)

def DTW_1(v1, v2, dist=euclidean):
    
    n, m = len(v1), len(v2)
    
    # initialize matrix
    D = np.zeros((n,m))
    D[1:,1:] = np.inf

    for i in range(n):
        for j in range(m):
            d_ij = dist(v1[i], v2[j])
            D[i,j] = d_ij + min(D[i-1,j], D[i,j-1], D[i-1,j-1])
    
    return D

def DTW(v1, v2, w=1, dist=euclidean):
    
    n, m = len(v1), len(v2)
    
    #adapt window size 
    w = max(w, abs(n-m))
    
    # initialize matrix
    D = np.zeros((n,m))
    D[:,:] = np.inf
    D[0,0] = 0
    
    for i in range(n):
        for j in range(max(0,i-w), min(m,i+w)):
            D[i,j] = 0

    for i in range(n):
        for j in range(max(0,i-w), min(m,i+w)):
            d_ij = dist(v1[i], v2[j])
            D[i,j] = d_ij + min(D[i-1,j], D[i,j-1], D[i-1,j-1])
    
    return D

In [ ]:
D = DTW(farr[0], farr[1], w=20)

plt.figure(figsize=(12,12))
im = plt.matshow(D, fignum=1, cmap='Blues')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('DTW Matrix', fontsize=20)
plt.show()

In [ ]:
D = DTW_1(farr[0], farr[1])

plt.figure(figsize=(12,12))
im = plt.matshow(D, fignum=1, cmap='Blues')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('DTW Matrix', fontsize=20)
plt.show()

In [ ]:
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import numpy as np

d, paths = dtw.warping_paths(farr[0], farr[1], window=50, psi=2)
best_path = dtw.best_path(paths)
dtwvis.plot_warpingpaths(farr[0], farr[1], paths, best_path)